In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [29]:
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [30]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model



In [31]:
import tensorflow as tf
print(tf.__version__)

2.17.1


In [32]:
pip install scikeras


Note: you may need to restart the kernel to use updated packages.


In [33]:
from scikeras.wrappers import KerasClassifier


In [34]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# make sure input_dim is defined e.g. input_dim = X.shape[1]
def build_model(neurons=32, layers=1, input_dim=None):
    model = Sequential()
    model.add(Dense(neurons, input_dim=input_dim, activation='relu'))
    for _ in range(layers - 1):
        model.add(Dense(neurons, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Note: pass model=build_model (SciKeras uses 'model' arg)
clf = KerasClassifier(model=build_model, verbose=0)

# Route model-building args with model__ prefix, fit args with fit__ prefix
param_grid = {
    'model__neurons': [16, 32, 64, 128],
    'model__layers': [1, 2],
    'fit__epochs': [50, 100,500],
    # you can also tune batch_size via fit__batch_size or set it in KerasClassifier(...)
}

grid = GridSearchCV(estimator=clf, param_grid=param_grid, n_jobs=-1, cv=3, verbose=1)


In [35]:
## Create a Keras classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

In [36]:

# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100,500]
}

In [37]:
import numpy as np
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

# 1. Update function to accept 'meta'
def create_model(neurons=32, layers=1, meta=None):
    # 2. Get input features from SciKeras metadata
    # This automatically detects X_train.shape[1]
    n_features_in_ = meta["n_features_in_"]

    model = Sequential()
    
    # 3. Use an explicit Input Layer
    # This forces Keras 3 to build the graph immediately, fixing the AttributeError
    model.add(Input(shape=(n_features_in_,)))
    
    # First hidden layer (no input_dim needed here anymore)
    model.add(Dense(neurons, activation='relu'))
    
    # Additional layers
    for _ in range(layers - 1):
        model.add(Dense(neurons, activation='relu'))
        
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Wrap model
# We do not need to pass input_shape manually here; SciKeras handles it via 'meta'
model = KerasClassifier(model=create_model, verbose=0)

param_grid = {
    'model__neurons': [16, 32, 64, 128],
    'model__layers': [1, 2],
    'fit__epochs': [50, 100, 500]
}

# Ensure data is numpy array and y is the correct shape
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

# Run Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=1)
grid_result = grid.fit(X_train, y_train)

print("Best Score: ", grid_result.best_score_)
print("Best Params:", grid_result.best_params_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
Best Score:  0.8571235891087507
Best Params: {'fit__epochs': 500, 'model__layers': 1, 'model__neurons': 16}
